In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
train_df=pd.read_csv('train.csv')

In [3]:
train_df.head()

,id,Doc_ID,Sent_ID,Word,tag
0,1,1,1,Obesity,O
1,2,1,1,in,O
2,3,1,1,Low-,O
3,4,1,1,and,O
4,5,1,1,Middle-Income,O


In [10]:
train_df['Word']=train_df['Word'].fillna(' ')

In [24]:
X = train_df.drop(['id','tag','Doc_ID'], axis=1)
v = DictVectorizer(sparse=True)
X = v.fit_transform(X.to_dict('records'))

In [25]:
y = train_df.tag.values

In [26]:
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
X_train.shape, y_train.shape

((3635066, 184507), (3635066,))

In [27]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [28]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-indications', 'I-indications']

In [29]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels=new_classes))

               precision    recall  f1-score   support

B-indications       0.64      0.45      0.53     10443
I-indications       0.61      0.37      0.46      8999

    micro avg       0.63      0.42      0.50     19442
    macro avg       0.63      0.41      0.50     19442
 weighted avg       0.63      0.42      0.50     19442



In [39]:
test_df=pd.read_csv('test.csv')

In [40]:
test_ids=test_df.id.values

In [41]:
test_df.head()

,id,Doc_ID,Sent_ID,Word
0,4543834,30001,191283,CCCVA
1,4543835,30001,191283,","
2,4543836,30001,191283,MANOVA
3,4543837,30001,191283,","
4,4543838,30001,191283,my


In [42]:
test_df=test_df.drop(columns=['id','Doc_ID'],axis=1)

In [43]:
test_df.head()

,Sent_ID,Word
0,191283,CCCVA
1,191283,","
2,191283,MANOVA
3,191283,","
4,191283,my


In [36]:
test_df['Word']=test_df['Word'].fillna(' ')

In [45]:
test_transform=v.transform(test_df.to_dict('records'))

In [46]:
test_preds=nb.predict(test_transform)

In [47]:
sub=pd.DataFrame()

In [170]:
sub['id']=test_ids
sub['Sent_ID']=test_df['Sent_ID']
sub['tag']=test_preds
sub.head()

,id,Sent_ID,tag
0,4543834,191283,O
1,4543835,191283,O
2,4543836,191283,O
3,4543837,191283,O
4,4543838,191283,O


In [49]:
sub.tag.value_counts()

O                2961633
B-indications      17827
I-indications      15003
Name: tag, dtype: int64

In [50]:
sub.to_csv('submission-multi-nb.csv',index=False)

In [53]:
!apt-get install zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 158 kB of archives.
After this operation, 587 kB of additional disk space will be used.
Get:1 http://us-east1.gce.archive.ubuntu.com/ubuntu xenial/main amd64 zip amd64 3.0-11 [158 kB]
Fetched 158 kB in 0s (5,533 kB/s)
Selecting previously unselected package zip.
(Reading database ... 76972 files and directories currently installed.)
Preparing to unpack .../archives/zip_3.0-11_amd64.deb ...
Unpacking zip (3.0-11) ...
Processing triggers for man-db (2.7.5-1) ...
Setting up zip (3.0-11) ...


In [54]:
!zip -r submission-multi-nb.zip submission-multi-nb.csv

  adding: submission-multi-nb.csv (deflated 73%)


### 10 fold - CV

In [59]:
from sklearn.model_selection import StratifiedKFold
import time

In [56]:
fold_n = 10
cv = fold_n   
folds = StratifiedKFold(n_splits=fold_n, random_state=10, shuffle=True)

In [92]:
y_pred_nb = {}
features = ['Sent_ID','Word']
oof = np.array(['O' for i in range(len(train_df))])

In [95]:
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())    
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    nb = MultinomialNB(alpha=0.01)
    nb.partial_fit(X_train, y_train, classes)
    oof[valid_index]=nb.predict(X_valid)
    y_pred_nb[fold_n]= nb.predict(test_transform)

Fold 0 started at Sat Mar 23 09:31:43 2019
Fold 1 started at Sat Mar 23 09:31:48 2019
Fold 2 started at Sat Mar 23 09:31:52 2019
Fold 3 started at Sat Mar 23 09:31:57 2019
Fold 4 started at Sat Mar 23 09:32:02 2019
Fold 5 started at Sat Mar 23 09:32:07 2019
Fold 6 started at Sat Mar 23 09:32:12 2019
Fold 7 started at Sat Mar 23 09:32:17 2019
Fold 8 started at Sat Mar 23 09:32:22 2019
Fold 9 started at Sat Mar 23 09:32:27 2019


In [73]:
cv_test_preds=pd.DataFrame(y_pred_nb)

In [76]:
cv_test_preds=cv_test_preds.replace('O',0)
cv_test_preds=cv_test_preds.replace('I-indications',1)
cv_test_preds=cv_test_preds.replace('B-indications',2)

In [77]:
cv_test_preds['final']=cv_test_preds.max(axis=1)

In [78]:
cv_test_preds['final'].value_counts()

0    2957477
2      20130
1      16856
Name: final, dtype: int64

In [79]:
cv_test_preds['final']=cv_test_preds['final'].replace(0,'O')
cv_test_preds['final']=cv_test_preds['final'].replace(1,'I-indications')
cv_test_preds['final']=cv_test_preds['final'].replace(2,'B-indications')

In [80]:
sub['tag']=cv_test_preds['final'].values

In [81]:
sub.to_csv('submission-multi-nb_cv.csv',index=False)

In [82]:
!zip -r submission-multi-nb_cv.zip submission-multi-nb_cv.csv

  adding: submission-multi-nb_cv.csv (deflated 73%)


### Eval metric

In [96]:
import ner_f1

In [99]:
ideal=pd.DataFrame()
ideal['id']=train_df['id']
ideal['Sent_ID']=train_df['Sent_ID']
ideal['tag']=y

In [100]:
pred=pd.DataFrame()
pred['id']=train_df['id']
pred['Sent_ID']=train_df['Sent_ID']
pred['tag']=oof

In [101]:
ner_f1.calculate_score(ideal,pred)

0.3009418906209923

### CRF

In [103]:
!pip3 install sklearn_crfsuite

    100% |████████████████████████████████| 747kB 871kB/s ta 0:00:011
You are using pip version 8.1.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [104]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [109]:
import spacy
pos_tag=[]
nlp = spacy.load('en_core_web_sm')

In [110]:
from tqdm import tqdm

In [114]:
unique_words=train_df['Word'].unique()

In [117]:
len(unique_words)

184506

In [ ]:
pos_tag={}
for ind in tqdm(range(len(unique_words))):
    doc=nlp(unique_words[ind])
    for token in doc:
        pos_tag[unique_words[ind]]=token.tag_


  2%|▏         | 3015/184506 [00:30<31:05, 97.29it/s]


  3%|▎         | 5961/184506 [01:00<29:42, 100.15it/s]


  5%|▍         | 8889/184506 [01:30<29:44, 98.40it/s]


  6%|▋         | 11771/184506 [01:59<29:53, 96.31it/s]


  8%|▊         | 14577/184506 [02:29<28:33, 99.17it/s]


...

 13%|█▎        | 23415/184506 [03:58<26:51, 99.97it/s]


 14%|█▍        | 26396/184506 [04:28<26:35, 99.13it/s]


 16%|█▌        | 29409/184506 [04:58<25:57, 99.57it/s] 


 18%|█▊        | 32379/184506 [05:27<24:55, 101.75it/s]


 18%|█▊        | 33390/184506 [05:38<24:58, 100.87it/s]

In [120]:
train_words=list(pos_tag.keys())
train_pos=list(pos_tag.values())

In [121]:
pos_df=pd.DataFrame()
pos_df['Word']=train_words
pos_df['pos']=train_pos

In [122]:
train_df_pos=train_df.merge(pos_df,on='Word',how='left')

In [124]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['pos'].values.tolist(), 
                                                           s['tag'].values.tolist())]
        self.grouped = self.data.groupby('Sent_ID').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(train_df_pos)
sentences = getter.sentences

In [1]:
sentences[0]

NameError: name 'sentences' is not defined

In [142]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features

In [143]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [144]:
def sent2labels(sent):
    return [label for token, postag, label in sent]

In [145]:
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [146]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [225]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=1000,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [248]:
from sklearn.metrics import make_scorer
import scipy.stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [249]:
labels = ['B-indications','I-indications']

In [250]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

OSError: [Errno 12] Cannot allocate memory

In [226]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

               precision    recall  f1-score   support

B-indications       0.89      0.80      0.84     10651
I-indications       0.86      0.81      0.83      8932

    micro avg       0.88      0.80      0.84     19583
    macro avg       0.87      0.80      0.84     19583
 weighted avg       0.88      0.80      0.84     19583



In [153]:
test_df['Word']=test_df['Word'].fillna(' ')

In [192]:
unique_words_test=test_df['Word'].unique()

In [193]:
pos_tag_test={}
for ind in tqdm(range(len(unique_words_test))):
    doc=nlp(unique_words_test[ind])
    for token in doc:
        pos_tag_test[unique_words_test[ind]]=token.tag_




  0%|          | 0/139892 [00:00<?, ?it/s]


  0%|          | 6/139892 [00:00<39:45, 58.64it/s]


  0%|          | 14/139892 [00:00<37:33, 62.07it/s]


  0%|          | 22/139892 [00:00<35:42, 65.30it/s]


  0%|          | 30/139892 [00:00<34:22, 67.82it/s]


  0%|          | 38/139892 [00:00<33:33, 69.45it/s]


  0%|          | 46/139892 [00:00<32:57, 70.73it/s]


  0%|          | 54/139892 [00:00<32:17, 72.18it/s]


  0%|          | 62/139892 [00:00<31:34, 73.80it/s]


  0%|          | 70/139892 [00:00<31:07, 74.85it/s]


  0%|          | 78/139892 [00:01<30:40, 75.97it/s]


  0%|          | 86/139892 [00:01<30:33, 76.26it/s]


  0%|          | 94/139892 [00:01<31:41, 73.50it/s]


  0%|          | 102/139892 [00:01<31:09, 74.78it/s]


  0%|          | 110/139892 [00:01<30:34, 76.19it/s]


  0%|          | 119/139892 [00:01<30:01, 77.60it/s]


  0%|          | 127/139892 [00:01<31:14, 74.55it/s]


  0%|          | 136/139892 [00:01<30:21, 76.74it/s]


  0%|          | 145/139892 [0

  1%|          | 1218/139892 [00:13<23:37, 97.84it/s]


  1%|          | 1228/139892 [00:13<23:37, 97.82it/s]


  1%|          | 1238/139892 [00:13<23:28, 98.42it/s]


  1%|          | 1248/139892 [00:13<23:29, 98.40it/s]


  1%|          | 1258/139892 [00:14<23:27, 98.47it/s]


  1%|          | 1268/139892 [00:14<23:34, 97.98it/s]


  1%|          | 1278/139892 [00:14<23:51, 96.83it/s]


  1%|          | 1288/139892 [00:14<23:53, 96.67it/s]


  1%|          | 1298/139892 [00:14<24:01, 96.17it/s]


  1%|          | 1309/139892 [00:14<23:41, 97.51it/s]


  1%|          | 1319/139892 [00:14<23:35, 97.88it/s]


  1%|          | 1329/139892 [00:14<23:37, 97.78it/s]


  1%|          | 1339/139892 [00:14<24:20, 94.85it/s]


  1%|          | 1349/139892 [00:14<24:05, 95.85it/s]


  1%|          | 1359/139892 [00:15<24:03, 95.94it/s]


  1%|          | 1369/139892 [00:15<24:04, 95.88it/s]


  1%|          | 1379/139892 [00:15<24:17, 95.06it/s]


  1%|          | 1389/139892 [00:15<24:25, 94.51

  3%|▎         | 3768/139892 [00:40<24:13, 93.65it/s]


  3%|▎         | 3778/139892 [00:40<23:51, 95.08it/s]


  3%|▎         | 3789/139892 [00:40<23:29, 96.58it/s]


  3%|▎         | 3799/139892 [00:40<23:19, 97.25it/s]


  3%|▎         | 3809/139892 [00:40<23:10, 97.90it/s]


  3%|▎         | 3820/139892 [00:40<22:55, 98.91it/s]


  3%|▎         | 3830/139892 [00:40<23:00, 98.59it/s]


  3%|▎         | 3841/139892 [00:40<22:49, 99.34it/s]


  3%|▎         | 3851/139892 [00:41<22:59, 98.58it/s]


  3%|▎         | 3861/139892 [00:41<23:12, 97.67it/s]


  3%|▎         | 3871/139892 [00:41<23:22, 96.98it/s]


  3%|▎         | 3881/139892 [00:41<23:32, 96.28it/s]


  3%|▎         | 3891/139892 [00:41<23:27, 96.62it/s]


  3%|▎         | 3901/139892 [00:41<24:14, 93.50it/s]


  3%|▎         | 3911/139892 [00:41<23:59, 94.45it/s]


  3%|▎         | 3921/139892 [00:41<23:46, 95.31it/s]


  3%|▎         | 3931/139892 [00:41<23:33, 96.20it/s]


  3%|▎         | 3941/139892 [00:42<23:38, 95.84

  4%|▍         | 6276/139892 [01:07<24:05, 92.43it/s]


  4%|▍         | 6286/139892 [01:07<23:53, 93.23it/s]


  5%|▍         | 6296/139892 [01:07<23:36, 94.30it/s]


  5%|▍         | 6306/139892 [01:07<23:33, 94.49it/s]


  5%|▍         | 6316/139892 [01:07<23:29, 94.80it/s]


  5%|▍         | 6326/139892 [01:07<23:25, 95.03it/s]


  5%|▍         | 6336/139892 [01:08<23:26, 94.97it/s]


  5%|▍         | 6346/139892 [01:08<23:21, 95.29it/s]


  5%|▍         | 6356/139892 [01:08<23:28, 94.84it/s]


  5%|▍         | 6366/139892 [01:08<23:33, 94.48it/s]


  5%|▍         | 6376/139892 [01:08<23:58, 92.82it/s]


  5%|▍         | 6386/139892 [01:08<23:51, 93.29it/s]


  5%|▍         | 6396/139892 [01:08<23:54, 93.07it/s]


  5%|▍         | 6406/139892 [01:08<23:49, 93.40it/s]


  5%|▍         | 6416/139892 [01:08<24:50, 89.54it/s]


  5%|▍         | 6426/139892 [01:08<24:17, 91.58it/s]


  5%|▍         | 6436/139892 [01:09<24:11, 91.92it/s]


  5%|▍         | 6446/139892 [01:09<23:53, 93.07

  6%|▋         | 8865/139892 [01:34<24:45, 88.21it/s]


  6%|▋         | 8874/139892 [01:34<25:00, 87.30it/s]


  6%|▋         | 8884/139892 [01:34<24:35, 88.79it/s]


  6%|▋         | 8894/139892 [01:34<24:12, 90.21it/s]


  6%|▋         | 8904/139892 [01:34<24:07, 90.49it/s]


  6%|▋         | 8914/139892 [01:35<24:01, 90.87it/s]


  6%|▋         | 8924/139892 [01:35<23:57, 91.13it/s]


  6%|▋         | 8934/139892 [01:35<23:45, 91.84it/s]


  6%|▋         | 8944/139892 [01:35<25:00, 87.28it/s]


  6%|▋         | 8954/139892 [01:35<24:27, 89.22it/s]


  6%|▋         | 8964/139892 [01:35<23:56, 91.15it/s]


  6%|▋         | 8974/139892 [01:35<23:47, 91.68it/s]


  6%|▋         | 8984/139892 [01:35<23:32, 92.67it/s]


  6%|▋         | 8994/139892 [01:35<23:39, 92.19it/s]


  6%|▋         | 9004/139892 [01:36<23:51, 91.43it/s]


  6%|▋         | 9014/139892 [01:36<23:51, 91.43it/s]


  6%|▋         | 9024/139892 [01:36<23:45, 91.79it/s]


  6%|▋         | 9034/139892 [01:36<23:47, 91.69

...

 93%|█████████▎| 129741/139892 [23:04<01:47, 94.02it/s]


 93%|█████████▎| 129751/139892 [23:04<01:47, 94.74it/s]


 93%|█████████▎| 129761/139892 [23:04<01:46, 94.72it/s]


 93%|█████████▎| 129772/139892 [23:04<01:44, 96.42it/s]


 93%|█████████▎| 129782/139892 [23:04<01:44, 96.40it/s]


 93%|█████████▎| 129792/139892 [23:04<01:47, 93.97it/s]


 93%|█████████▎| 129802/139892 [23:04<01:47, 94.00it/s]


 93%|█████████▎| 129812/139892 [23:04<01:47, 93.92it/s]


 93%|█████████▎| 129822/139892 [23:04<01:47, 94.03it/s]


 93%|█████████▎| 129832/139892 [23:05<01:46, 94.31it/s]


 93%|█████████▎| 129842/139892 [23:05<01:46, 94.30it/s]


 93%|█████████▎| 129852/139892 [23:05<01:48, 92.79it/s]


 93%|█████████▎| 129862/139892 [23:05<01:47, 93.22it/s]


 93%|█████████▎| 129872/139892 [23:05<01:47, 92.82it/s]


 93%|█████████▎| 129882/139892 [23:05<01:47, 92.98it/s]


 93%|█████████▎| 129892/139892 [23:05<01:46, 94.06it/s]


 93%|█████████▎| 129902/139892 [23:05<01:46, 94.06it/s]


 93%|█████████

 94%|█████████▍| 132182/139892 [23:29<01:21, 95.17it/s]


 94%|█████████▍| 132192/139892 [23:30<01:21, 94.67it/s]


 95%|█████████▍| 132202/139892 [23:30<01:21, 94.79it/s]


 95%|█████████▍| 132212/139892 [23:30<01:20, 95.49it/s]


 95%|█████████▍| 132222/139892 [23:30<01:20, 95.85it/s]


 95%|█████████▍| 132232/139892 [23:30<01:25, 89.57it/s]


 95%|█████████▍| 132242/139892 [23:30<01:23, 91.17it/s]


 95%|█████████▍| 132252/139892 [23:30<01:22, 92.90it/s]


 95%|█████████▍| 132262/139892 [23:30<01:21, 93.79it/s]


 95%|█████████▍| 132272/139892 [23:30<01:20, 94.43it/s]


 95%|█████████▍| 132282/139892 [23:31<01:20, 95.04it/s]


 95%|█████████▍| 132292/139892 [23:31<01:20, 94.06it/s]


 95%|█████████▍| 132302/139892 [23:31<01:20, 94.37it/s]


 95%|█████████▍| 132312/139892 [23:31<01:19, 95.15it/s]


 95%|█████████▍| 132322/139892 [23:31<01:18, 96.51it/s]


 95%|█████████▍| 132332/139892 [23:31<01:18, 96.11it/s]


 95%|█████████▍| 132342/139892 [23:31<01:22, 91.49it/s]


 95%|█████████

 96%|█████████▌| 134625/139892 [23:55<00:55, 94.59it/s]


 96%|█████████▌| 134635/139892 [23:55<00:55, 95.13it/s]


 96%|█████████▌| 134645/139892 [23:55<00:54, 96.17it/s]


 96%|█████████▋| 134655/139892 [23:55<00:54, 96.02it/s]


 96%|█████████▋| 134665/139892 [23:56<00:54, 95.95it/s]


 96%|█████████▋| 134675/139892 [23:56<00:54, 95.44it/s]


 96%|█████████▋| 134685/139892 [23:56<00:54, 94.71it/s]


 96%|█████████▋| 134695/139892 [23:56<00:55, 94.36it/s]


 96%|█████████▋| 134705/139892 [23:56<00:54, 95.05it/s]


 96%|█████████▋| 134715/139892 [23:56<00:54, 95.63it/s]


 96%|█████████▋| 134725/139892 [23:56<00:53, 96.12it/s]


 96%|█████████▋| 134735/139892 [23:56<00:53, 96.08it/s]


 96%|█████████▋| 134745/139892 [23:56<00:53, 96.06it/s]


 96%|█████████▋| 134755/139892 [23:56<00:53, 95.98it/s]


 96%|█████████▋| 134765/139892 [23:57<00:53, 96.07it/s]


 96%|█████████▋| 134775/139892 [23:57<00:54, 94.04it/s]


 96%|█████████▋| 134785/139892 [23:57<00:54, 94.25it/s]


 96%|█████████

 98%|█████████▊| 137066/139892 [24:21<00:30, 93.24it/s]


 98%|█████████▊| 137076/139892 [24:21<00:30, 92.42it/s]


 98%|█████████▊| 137086/139892 [24:21<00:30, 93.30it/s]


 98%|█████████▊| 137096/139892 [24:21<00:29, 94.56it/s]


 98%|█████████▊| 137106/139892 [24:21<00:29, 94.83it/s]


 98%|█████████▊| 137116/139892 [24:21<00:29, 94.76it/s]


 98%|█████████▊| 137126/139892 [24:21<00:29, 94.89it/s]


 98%|█████████▊| 137136/139892 [24:21<00:28, 95.21it/s]


 98%|█████████▊| 137146/139892 [24:22<00:28, 95.09it/s]


 98%|█████████▊| 137156/139892 [24:22<00:28, 94.85it/s]


 98%|█████████▊| 137166/139892 [24:22<00:28, 95.26it/s]


 98%|█████████▊| 137176/139892 [24:22<00:28, 94.46it/s]


 98%|█████████▊| 137186/139892 [24:22<00:28, 94.78it/s]


 98%|█████████▊| 137196/139892 [24:22<00:28, 95.47it/s]


 98%|█████████▊| 137206/139892 [24:22<00:28, 95.21it/s]


 98%|█████████▊| 137216/139892 [24:22<00:28, 95.40it/s]


 98%|█████████▊| 137226/139892 [24:22<00:27, 95.51it/s]


 98%|█████████

100%|█████████▉| 139418/139892 [24:46<00:04, 98.88it/s]


100%|█████████▉| 139428/139892 [24:47<00:04, 98.58it/s]


100%|█████████▉| 139438/139892 [24:47<00:04, 97.91it/s]


100%|█████████▉| 139448/139892 [24:47<00:04, 97.05it/s]


100%|█████████▉| 139458/139892 [24:47<00:04, 95.05it/s]


100%|█████████▉| 139468/139892 [24:47<00:04, 95.05it/s]


100%|█████████▉| 139478/139892 [24:47<00:04, 95.45it/s]


100%|█████████▉| 139488/139892 [24:47<00:04, 95.73it/s]


100%|█████████▉| 139498/139892 [24:47<00:04, 96.60it/s]


100%|█████████▉| 139508/139892 [24:47<00:03, 97.52it/s]


100%|█████████▉| 139518/139892 [24:48<00:03, 97.02it/s]


100%|█████████▉| 139528/139892 [24:48<00:03, 96.44it/s]


100%|█████████▉| 139538/139892 [24:48<00:03, 95.60it/s]


100%|█████████▉| 139548/139892 [24:48<00:03, 95.42it/s]


100%|█████████▉| 139558/139892 [24:48<00:03, 95.98it/s]


100%|█████████▉| 139568/139892 [24:48<00:03, 96.16it/s]


100%|█████████▉| 139578/139892 [24:48<00:03, 96.54it/s]


100%|█████████

In [194]:
test_words=list(pos_tag_test.keys())
test_pos=list(pos_tag_test.values())

In [195]:
pos_df_test=pd.DataFrame()
pos_df_test['Word']=test_words
pos_df_test['pos']=test_pos

In [196]:
test_df_pos=test_df.merge(pos_df_test,on='Word',how='left')

In [197]:
test_df_pos['tag']='O'

In [198]:
#test_df_pos['pos']=test_df_pos['pos'].fillna('NN')

In [199]:
test_getter = SentenceGetter(test_df_pos)
test_sentences = test_getter.sentences

In [200]:
test_df_X = [sent2features(s) for s in test_sentences]
test_df_y = [sent2labels(s) for s in test_sentences]

In [227]:
y_test_pred = crf.predict(test_df_X)

In [228]:
len(y_test_pred)

125840

In [229]:
y_test_pred_unpack=[val for sub_li in y_test_pred for val in sub_li]

In [230]:
len(y_test_pred_unpack)

2994463

In [231]:
len(test_df)

2994463

In [232]:
sub['tag']=y_test_pred_unpack

In [233]:
sub.head()

,id,Sent_ID,tag
0,4543834,191283,O
1,4543835,191283,O
2,4543836,191283,O
3,4543837,191283,O
4,4543838,191283,O


In [234]:
sub.to_csv('submission-crf_1000_iter.csv',index=False)

In [237]:
import gc
gc.collect()

47

In [238]:
!zip -r submission-crf_iter.zip submission-crf_1000_iter.csv

OSError: [Errno 12] Cannot allocate memory

In [210]:
sub.tag.value_counts()

O                2937610
B-indications      29821
I-indications      27032
Name: tag, dtype: int64

### 10 fold CV

X,y,test_df_X are the values we have to use

In [218]:
from sklearn.model_selection import KFold

In [219]:
kf=KFold(n_splits=2, random_state=None, shuffle=True)

In [211]:
y_pred_crf = {}
oof_crf = np.array(['O' for i in range(len(train_df))])

In [223]:
for fold_n, (train_index, valid_index) in enumerate(kf.split(X)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train=[]
    X_valid=[]
    y_train=[]
    y_valid=[]
    for ind in train_index:
        X_train.append(X[ind])
        y_train.append(y[ind])
    for ind in valid_index:
        X_valid.append(X[ind])
        y_valid.append(y[ind])
    crf = sklearn_crfsuite.CRF(
                algorithm='lbfgs',
                c1=0.1,
                c2=0.1,
                max_iterations=100,
                all_possible_transitions=True
            )
    crf.fit(X_train, y_train)
    #oof_crf[valid_index]=crf.predict(X_valid)
    y_pred_crf[fold_n]= crf.predict(test_df_X)

Fold 0 started at Sat Mar 23 20:06:32 2019
Fold 1 started at Sat Mar 23 20:12:47 2019


In [224]:
len(y_pred_crf)

2

In [252]:
import pickle
with open('train_x.pkl', 'wb') as f:
    pickle.dump(X, f)
with open('train_y.pkl', 'wb') as f:
    pickle.dump(y, f)
with open('test.pkl', 'wb') as f:
    pickle.dump(test_df_X, f)